In [ ]:
import pandas as pd
text = pd.read_csv('/content/drive/MyDrive/Social Media Group /Data/posts_test.csv')

In [ ]:
price = pd.read_csv('/content/drive/MyDrive/Social Media Group /Data/WRDS/close&eps_171819.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
ticker = pd.read_csv('/content/drive/MyDrive/Social Media Group /Data/Train_result/ticker.csv')

In [ ]:
ticker = ticker['tic'].tolist()

# get 3 day change and close price lag 1

In [ ]:
### get percent change of close price
def percent_change(price_twitter,tic,day_diff,lag):
    #try stock: RMBS
    #tic = "RMBS"
    TIC_price = price_twitter.loc[price_twitter['tic']==tic].copy()

    #set n_day_change
    #day_diff = 3

    ### get percent change
    tmp = list()
    for index in range(len(TIC_price)):
        if index+day_diff >= len(TIC_price):
            # the first few data point has no n_day_change
            tmp.append(-100000000)
        else:
            today_price = TIC_price['prccd'].iloc[index]
            future_price = TIC_price['prccd'].iloc[index+day_diff]
            diff_percent = (future_price - today_price)/today_price
            tmp.append(diff_percent)
        
    #this is the column name of the new n_day_change column
    col_name = str(day_diff)+'_day_change'
    #join
    TIC_price[col_name] = tmp 
    
    
    ### get lag close price
    tmp = list()
    for index in range(len(TIC_price)):
        if index < lag:
            # the first few data point has no n_day_change
            tmp.append(-100000000)
        else:
            yesterday_price = TIC_price['prccd'].iloc[index-lag]
            tmp.append(yesterday_price)
    
    #this is the column name of the new n_day_change column
    col_name = 'close_price_lag_'+str(lag)
    #join
    TIC_price[col_name] = tmp 
    
    return TIC_price

In [ ]:
from tqdm import tqdm
# get n_day_change for all stocks, store in a dict
price_pchange = dict()
for tic in tqdm(ticker):
    # get the n_day_change filter by tic
    TIC_price = percent_change(price,tic,3,1)
    # store into the dictionary
    price_pchange[tic] = TIC_price

100%|██████████| 1765/1765 [03:06<00:00,  9.47it/s]


In [ ]:
list(price_pchange[tic].columns)

['gvkey',
 'iid',
 'datadate',
 'tic',
 'cshoc',
 'cshtrd',
 'eps',
 'prccd',
 '3_day_change',
 'close_price_lag_1']

In [ ]:
output = pd.DataFrame(columns = list(price_pchange[tic].columns))

for tic in tqdm(ticker):
    tmp = price_pchange[tic]
    output = output.append(tmp)

100%|██████████| 1765/1765 [02:35<00:00, 11.34it/s]


In [ ]:
price = output[output['3_day_change'] != -100000000]

# matching

In [ ]:
# datetime transform
price['datadate'] = pd.to_datetime(price['datadate'],format='%Y%m%d')
text['created_time'] = pd.to_datetime(text['created_time'],format='%Y-%m-%d')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
text['datadate'] = text['created_time']
text['tic'] = text['ticker']
text['tic'] = text['tic'].apply(lambda x: x[1:])
text = text.drop(columns = ['Unnamed: 0','created_time','ticker'])

In [ ]:
# price data after 2019-06-01
from datetime import datetime
date = datetime.strptime("2019-06-01", '%Y-%m-%d')
price = price[price['datadate'] >= date]

In [ ]:
text = text.set_index(['tic','datadate'])
price = price.set_index(['tic','datadate'])

In [ ]:
# match data
match_df = price.join(text)

In [ ]:
match_df = match_df.reset_index()
match_df.isnull().sum()

tic                      0
datadate                 0
gvkey                    0
iid                      0
cshoc                    0
cshtrd                   1
eps                   4712
prccd                    0
3_day_change             0
close_price_lag_1        0
text                 41115
retweets             41115
replies              41115
likes                41115
is_retweet           41115
rank                 41115
dtype: int64

In [ ]:
match_df

,tic,datadate,gvkey,iid,cshoc,cshtrd,eps,prccd,3_day_change,close_price_lag_1,text,retweets,replies,likes,is_retweet,rank
0,PLUG,2019-06-03,125604,01,230412000.0,3837351.0,-0.42,2.59,-0.011583,2.56,plug plug pic twitter comdohvkfsgee plug power...,1.0,2.0,5.0,1.0,2.0
1,PLUG,2019-06-04,125604,01,230412000.0,2880411.0,-0.42,2.58,-0.011628,2.59,gensure control backup power increase reliabil...,0.0,3.0,4.0,0.0,3.0
2,PLUG,2019-06-05,125604,01,230412000.0,3120313.0,-0.42,2.54,0.047244,2.58,plug hanging thread plug maybe one day uber ly...,0.0,2.0,6.0,1.0,4.0
3,PLUG,2019-06-06,125604,01,230412000.0,2562470.0,-0.42,2.56,0.023438,2.54,hmm big question plug amzn collaboration team ...,0.0,7.0,19.0,0.0,5.0
4,PLUG,2019-06-07,125604,01,230412000.0,2202802.0,-0.42,2.55,0.019608,2.56,remember company plug fslr meyer burger plug n...,0.0,3.0,3.0,1.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255920,ARL,2019-12-19,1545,03,15997000.0,994.0,7.73,15.56,0.008355,15.12,short ratio cto arl arl major change stock pri...,0.0,0.0,0.0,0.0,138.0
255921,ARL,2019-12-20,1545,03,15997000.0,5506.0,7.73,15.14,0.076618,15.56,arl move significant price change,0.0,0.0,0.0,0.0,139.0
255922,ARL,2019-12-23,1545,03,15997000.0,6027.0,7.73,15.43,0.088140,15.14,arl major change stock price change recently,0.0,0.0,0.0,0.0,142.0
255923,ARL,2019-12-24,1545,03,15997000.0,2920.0,7.73,15.69,0.065806,15.43,arl significant price movement stock change re...,0.0,0.0,0.0,0.0,143.0


In [ ]:
# matching the training data tickers

In [ ]:
match_df['in'] = match_df['tic'].apply(lambda x: 1 if x in ticker else 0)

In [ ]:
match_df = match_df[match_df['in'] == 1]
match_df.columns

Index(['tic', 'datadate', 'gvkey', 'iid', 'cshoc', 'cshtrd', 'eps', 'prccd',
       '3_day_change', 'close_price_lag_1', 'text', 'retweets', 'replies',
       'likes', 'is_retweet', 'rank', 'in'],
      dtype='object')

In [ ]:
match_df = match_df.drop(columns = ['gvkey', 'iid','in'])
match_df = match_df.reset_index().drop(columns = 'index')

In [ ]:
match_df = match_df.drop(columns = 'rank')
match_df

,tic,datadate,cshoc,cshtrd,eps,prccd,3_day_change,close_price_lag_1,text,retweets,replies,likes,is_retweet
0,PLUG,2019-06-03,230412000.0,3837351.0,-0.42,2.59,-0.011583,2.56,plug plug pic twitter comdohvkfsgee plug power...,1.0,2.0,5.0,1.0
1,PLUG,2019-06-04,230412000.0,2880411.0,-0.42,2.58,-0.011628,2.59,gensure control backup power increase reliabil...,0.0,3.0,4.0,0.0
2,PLUG,2019-06-05,230412000.0,3120313.0,-0.42,2.54,0.047244,2.58,plug hanging thread plug maybe one day uber ly...,0.0,2.0,6.0,1.0
3,PLUG,2019-06-06,230412000.0,2562470.0,-0.42,2.56,0.023438,2.54,hmm big question plug amzn collaboration team ...,0.0,7.0,19.0,0.0
4,PLUG,2019-06-07,230412000.0,2202802.0,-0.42,2.55,0.019608,2.56,remember company plug fslr meyer burger plug n...,0.0,3.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
255920,ARL,2019-12-19,15997000.0,994.0,7.73,15.56,0.008355,15.12,short ratio cto arl arl major change stock pri...,0.0,0.0,0.0,0.0
255921,ARL,2019-12-20,15997000.0,5506.0,7.73,15.14,0.076618,15.56,arl move significant price change,0.0,0.0,0.0,0.0
255922,ARL,2019-12-23,15997000.0,6027.0,7.73,15.43,0.088140,15.14,arl major change stock price change recently,0.0,0.0,0.0,0.0
255923,ARL,2019-12-24,15997000.0,2920.0,7.73,15.69,0.065806,15.43,arl significant price movement stock change re...,0.0,0.0,0.0,0.0


In [ ]:
match_df[match_df['close_price_lag_1'] == -100000000]

,tic,datadate,cshoc,cshtrd,eps,prccd,3_day_change,close_price_lag_1,text,retweets,replies,likes,is_retweet


In [ ]:
match_df.to_csv('/content/drive/MyDrive/Social Media Group /Data/topic_modeling_test.csv')

In [ ]:
pd.read_csv('/content/drive/MyDrive/Social Media Group /Data/topic_modeling_test.csv')

,Unnamed: 0,tic,datadate,cshoc,cshtrd,eps,prccd,3_day_change,close_price_lag_1,text,retweets,replies,likes,is_retweet
0,0,PLUG,2019-06-03,230412000.0,3837351.0,-0.42,2.59,-0.011583,2.56,plug plug pic twitter comdohvkfsgee plug power...,1.0,2.0,5.0,1.0
1,1,PLUG,2019-06-04,230412000.0,2880411.0,-0.42,2.58,-0.011628,2.59,gensure control backup power increase reliabil...,0.0,3.0,4.0,0.0
2,2,PLUG,2019-06-05,230412000.0,3120313.0,-0.42,2.54,0.047244,2.58,plug hanging thread plug maybe one day uber ly...,0.0,2.0,6.0,1.0
3,3,PLUG,2019-06-06,230412000.0,2562470.0,-0.42,2.56,0.023438,2.54,hmm big question plug amzn collaboration team ...,0.0,7.0,19.0,0.0
4,4,PLUG,2019-06-07,230412000.0,2202802.0,-0.42,2.55,0.019608,2.56,remember company plug fslr meyer burger plug n...,0.0,3.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255920,255920,ARL,2019-12-19,15997000.0,994.0,7.73,15.56,0.008355,15.12,short ratio cto arl arl major change stock pri...,0.0,0.0,0.0,0.0
255921,255921,ARL,2019-12-20,15997000.0,5506.0,7.73,15.14,0.076618,15.56,arl move significant price change,0.0,0.0,0.0,0.0
255922,255922,ARL,2019-12-23,15997000.0,6027.0,7.73,15.43,0.088140,15.14,arl major change stock price change recently,0.0,0.0,0.0,0.0
255923,255923,ARL,2019-12-24,15997000.0,2920.0,7.73,15.69,0.065806,15.43,arl significant price movement stock change re...,0.0,0.0,0.0,0.0
